In [1]:
# import all we will need
from typing import Protocol
from collections import deque

import random
from collections import deque
import pygame

pygame 2.6.1 (SDL 2.28.4, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# the game class

class SnakeGame:
    def __init__(self, xsize: int=30, ysize: int=30, scale: int=15):
        self.grid = Vector(xsize, ysize)
        self.scale = scale
        self.snake = Snake(game=self)
        self.food = Food(game=self)

    def run(self):
        running = True
        while running:
            next_move = self.controller.update()
            if next_move: self.snake.v = next_move
            self.snake.move()
            
            if not self.snake.p.within(self.grid):
                running = False
                message = 'Game over! You crashed into the wall!'
            if self.snake.cross_own_tail:
                running = False
                message = 'Game over! You hit your own tail!'
            if self.snake.p == self.food.p:
                self.snake.add_score()
                self.food = Food(game=self)
        print(f'{message} ... Score: {self.snake.score} , alive: {self.snake.moves} , Turns: {self.snake.turns}')


In [3]:
# smaller classes

class Food:
    def __init__(self, game: SnakeGame):
        self.game = game
        self.p = Vector.random_within(self.game.grid)

class Vector:
    def __init__(self, x: int=0, y: int=0):
        self.x = x
        self.y = y

    def __str__(self):
        return f'Vector({self.x}, {self.y})'

    def __add__(self, other: 'Vector') -> 'Vector':
        return Vector(self.x + other.x, self.y + other.y)

    def within(self, scope: 'Vector') -> 'Vector':
        return self.x <= scope.x and self.x >= 0 and self.y <= scope.y and self.y >= 0

    def __eq__(self, other: 'Vector') -> bool:
        return self.x == other.x and self.y == other.y

    @classmethod
    def random_within(cls, scope: 'Vector') -> 'Vector':
        return Vector(random.randint(0, scope.x - 1), random.randint(0, scope.y - 1))


class Snake:
    def __init__(self, *, game: SnakeGame):
        self.game = game
        self.score = 0
        self.moves = 0 # self added -----------------------------------
        self.turns = 0 # self added -----------------------------------
        self.v = Vector(0, 0)
        self.body = deque()
        self.body.append(Vector.random_within(self.game.grid))

    def move(self):
        self.p = self.p + self.v
        self.moves += 1

    @property
    def cross_own_tail(self):
        try:
            self.body.index(self.p, 1)
            return True
        except ValueError:
            return False

    @property
    def p(self):
        return self.body[0]

    @p.setter
    def p(self, value):
        self.body.appendleft(value)
        self.body.pop()

    def add_score(self):
        self.score += 1
        tail = self.body.pop()
        self.body.append(tail)
        self.body.append(tail)

    def debug(self):
        print('===')
        for i in self.body:
            print(str(i))



In [4]:
# controllers

class GameController(Protocol):
    def update(self) -> Vector:
        pass


class HumanController(GameController):
    def __init__(self, game):
        self.game = game
        self.game.controller = self
        pygame.init()
        self.screen = pygame.display.set_mode((game.grid.x * game.scale, game.grid.y * game.scale))
        self.clock = pygame.time.Clock()

        self.color_snake_head = (0, 255, 0)
        self.color_food = (255, 0, 0)

    def __del__(self):
        pygame.quit()

    def update(self) -> Vector:
        next_move = None
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    next_move = Vector(-1, 0)
                    self.game.snake.turns += 1 # self added --------------------
                if event.key == pygame.K_RIGHT:
                    next_move = Vector(1, 0)
                    self.game.snake.turns += 1 # self added --------------------
                if event.key == pygame.K_UP:
                    next_move = Vector(0, -1)
                    self.game.snake.turns += 1 # self added --------------------
                if event.key == pygame.K_DOWN:
                    next_move = Vector(0, 1)
                    self.game.snake.turns += 1 # self added --------------------
        self.screen.fill('black')
        for i, p in enumerate(self.game.snake.body):
            pygame.draw.rect(self.screen,
                                (0, max(128, 255 - i * 12), 0),
                                self.block(p))
        pygame.draw.rect(self.screen, self.color_food, self.block(self.game.food.p))
        pygame.display.flip()
        self.clock.tick(10)
        return next_move

    def block(self, obj):
        return (obj.x * self.game.scale,
                obj.y * self.game.scale,
                self.game.scale,
                self.game.scale)


In [5]:
# run the game

if __name__ == '__main__':
    game = SnakeGame()
    controller = HumanController(game)
    game.run()


2025-04-22 11:43:39.442 python3[37167:245913] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-22 11:43:39.442 python3[37167:245913] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Game over! You hit your own tail! ... Score: 6 , alive: 159 , Turns: 19
